In [5]:
import torch
import torch.nn as nn
from BNN_01 import *
import numpy as np
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Function

In [6]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root='/home/ygl/code', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='/home/ygl/code', train=False, transform=transform, download=True)

In [7]:
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = BinaryConv2d(1, 6, kernel_size=3, stride=1, padding=0, bias=False)
        self.conv2 = BinaryConv2d(6, 12, kernel_size=3, stride=1, padding=0, bias=False)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = BinaryLinear(12*12*12, 10, bias=False)
        # self.relu3 = nn.ReLU()
        # self.fc2 = BinaryLinear(120, 84, bias=False)
        # self.relu4 = nn.ReLU()
        # self.fc3 = BinaryLinear(84, num_classes, bias=False)
    def forward(self, x):
        #大于127的变成1，小于等于127的变成0
        x = torch.where(x>0.5, torch.tensor(1.), torch.tensor(0.))
        x = self.conv1(x)
        #print(x.int())
        #x = self.pool1(x)
        #print(x.int())
        x = torch.where(x>0, torch.tensor(1.), torch.tensor(0.))
        #print(x.int())
        x = self.conv2(x)
        #print(x.int())
        #x = self.pool2(x)
        #x = self.relu2(x)
        #print(x.int())
        x = self.pool2(x)
        x = torch.where(x>0, torch.tensor(1.), torch.tensor(0.))
        #x = self.conv3(x)
        #x = self.pool3(x)
        #x = torch.where(x>0, torch.tensor(1.), torch.tensor(-1.))
        #print(x.int())
        x = x.view(x.size(0), -1)
        #print(x.int())
        x = self.fc1(x)
        # x = self.relu3(x)
        # x = self.fc2(x)
        # x = self.relu3(x)
        # x = self.fc3(x)
        return x

In [9]:
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)
net = LeNet()
from tqdm import tqdm
np.int = int
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
EPOCH = 10
device = torch.device('cuda:0')
net.to(device)
for epoch in range(EPOCH):
    net.train()
    for i ,data in enumerate(dataloader):
        inputs, labels = data
        inputs = inputs.to(device).cuda()
        labels = labels.to(device).cuda()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        functional.reset_net(net)
        if i % 100 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, loss.item()))
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i,data in enumerate(testloader):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
torch.save(net.state_dict(), 'weight/lenet_01.pth')

[1,     1] loss: 2.321


[1,   101] loss: 0.780
[1,   201] loss: 0.831
[1,   301] loss: 0.564
[1,   401] loss: 0.694
Accuracy of the network on the 10000 test images: 80 %
[2,     1] loss: 0.505
[2,   101] loss: 0.553
[2,   201] loss: 0.516
[2,   301] loss: 0.483
[2,   401] loss: 0.367
Accuracy of the network on the 10000 test images: 82 %
[3,     1] loss: 0.614
[3,   101] loss: 0.534
[3,   201] loss: 0.649
[3,   301] loss: 0.572
[3,   401] loss: 0.568
Accuracy of the network on the 10000 test images: 81 %
[4,     1] loss: 0.542
[4,   101] loss: 0.464
[4,   201] loss: 0.582
[4,   301] loss: 0.572
[4,   401] loss: 0.434
Accuracy of the network on the 10000 test images: 81 %
[5,     1] loss: 0.585
[5,   101] loss: 0.559
[5,   201] loss: 0.551
[5,   301] loss: 0.539
[5,   401] loss: 0.532
Accuracy of the network on the 10000 test images: 83 %
[6,     1] loss: 0.526
[6,   101] loss: 0.584
[6,   201] loss: 0.564
[6,   301] loss: 0.718
[6,   401] loss: 0.468
Accuracy of the network on the 10000 test images: 82 %
[7,